# Postmortem experiment
Purpose: to see whether the same results can be obtained by transforming the target variable the same way as in the sklift ClassTransformation model but without using the sklift. This is necessary for planned use of keras for uplift prediction.

# Model

In [61]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklift.models import ClassTransformation
from sklift import metrics
from sklearn.metrics import roc_auc_score

In [2]:
best_params_1 = {'estimator__learning_rate': 0.05,
 'estimator__max_depth': 3,
 'estimator__n_estimators': 25,
 'estimator__reg_alpha': 0.05,
 'estimator__reg_lambda': 0}

In [3]:
sorted_features = ['first_redeem_date', 
                   'monthly_spend_mean', 
                   'express_points_spent_sum', 
                   'express_points_spent_last_month', 
                   'express_points_spent_mean', 
                   'regular_points_spent_mean',
                   'age', 
                   'days_since_last_transact',
                   'redeem_delay',
                   'regular_points_received_last_month',
                   'nunique',
                   'first_issue_date',
                   'last_month_spend',
                   'purchase_sum',
                   'purchase_mean', 
                   'regular_points_received_mean',
                   'regular_points_received_sum',
                   'relative_spend_last_month']

In [4]:
best_features = sorted_features[0:13]

In [6]:
df_full = pd.read_csv('train.csv', index_col='client_id')\
    .join(pd.read_csv('clients_train_modified_2.csv', index_col='client_id'), how='left')
df_full

,treatment_flg,purchased,regular_points_received_mean,regular_points_received_sum,express_points_received_mean,express_points_received_sum,regular_points_spent_mean,regular_points_spent_sum,express_points_spent_mean,express_points_spent_sum,...,last_month_spend,relative_spend_last_month,regular_points_received_last_month,express_points_received_last_month,regular_points_spent_last_month,express_points_spent_last_month,nunique,bought_most_pop,days_since_last_transact,purchased_own_products
client_id,,,,,,,,,,,,,,,,,,,,,
ad6561e2d8,1,1,2.432500,97.3,0.000000,0.0,-1.600000,-64.0,0.000000,0.0,...,2721.15,0.965714,1.966667,0.0,0.0,0.0,40,1.0,2,1.0
7c1ccbf93f,1,1,4.700000,9.4,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,851.00,0.900000,4.700000,0.0,0.0,0.0,2,1.0,13,1.0
b58fadcab6,1,1,2.772727,91.5,0.000000,0.0,-9.363636,-309.0,0.000000,0.0,...,2695.44,0.735432,3.050000,0.0,0.0,0.0,33,1.0,2,1.0
e99e6fabb9,0,0,1.046154,13.6,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,927.00,1.140460,1.100000,0.0,0.0,0.0,13,0.0,3,1.0
27fb6f8520,1,1,2.740000,82.2,0.666667,20.0,-0.133333,-4.0,-1.666667,-50.0,...,4200.60,1.487132,4.137500,0.0,0.0,0.0,30,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999d284453,1,1,8.625000,34.5,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,471.00,0.611551,2.300000,0.0,0.0,0.0,4,0.0,3,1.0
f634deea4e,0,1,1.086792,57.6,0.000000,0.0,-1.754717,-93.0,0.000000,0.0,...,3535.78,1.296013,1.088889,0.0,0.0,0.0,53,1.0,0,1.0
16cb4f99b0,0,1,3.485714,24.4,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,600.00,0.700637,1.450000,0.0,0.0,0.0,7,0.0,3,1.0


Transformation of the target variable

In [9]:
df_full['z'] =\
    df_full.purchased * df_full.treatment_flg + (1 - df_full.purchased) * (1 - df_full.treatment_flg)

Checking

In [10]:
df_full[['purchased', 'treatment_flg', 'z']]

,purchased,treatment_flg,z
client_id,,,
ad6561e2d8,1,1,1
7c1ccbf93f,1,1,1
b58fadcab6,1,1,1
e99e6fabb9,0,0,1
27fb6f8520,1,1,1
...,...,...,...
999d284453,1,1,1
f634deea4e,1,0,0
16cb4f99b0,1,0,0


In [11]:
X = df_full.loc[:, best_features]
y = df_full.purchased
treat = df_full.treatment_flg
z = df_full.z

In [21]:
X_train, X_test, y_train, y_test, treat_train, treat_test, z_train, z_test = \
    train_test_split(X, y, treat, z,
                     test_size=0.3,
                     random_state=12345,
                     shuffle=True)

# Prediction using the ClassTransformation function

In [23]:
estimator = XGBClassifier(random_state=8, 
                          use_label_encoder=False)

In [24]:
model_1 = ClassTransformation(estimator = estimator)

In [25]:
model_1.set_params(**best_params_1)

ClassTransformation(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05,
                                            max_delta_step=None, max_depth=3,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=25, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=8,
                                            reg_alpha=0.05, reg_lambda=0,
                                            scale_pos_weight=None,
                                            subsample=None, tree_method=None,
                                            use_label_encoder=False,
                                            validate_parameters=None,
                                            verbosity=None))

In [26]:
model_1 = model_1.fit(X_train, y_train, treat_train)

/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[14:56:29] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [27]:
y_pred_1 = model_1.predict(X_test)

In [37]:
y_pred_1

array([0.05374074, 0.11951017, 0.01691854, ..., 0.02113307, 0.01366198,
       0.014274  ], dtype=float32)

In [28]:
metrics.qini_auc_score(y_test, y_pred_1, treat_test)

0.020093042527387844

In [63]:
metrics.uplift_auc_score(y_test, y_pred_1, treat_test)

0.02861186692595083

# Prediction with the transformed variable

In [48]:
best_params_2 = {'learning_rate': 0.05,
 'max_depth': 3,
 'n_estimators': 25,
 'reg_alpha': 0.05,
 'reg_lambda': 0}

In [49]:
model_2 = XGBClassifier(random_state=8, 
                        use_label_encoder=False,
                       **best_params_2)

In [24]:
model_2 = ClassTransformation(estimator = estimator)

In [25]:
model_1.set_params(**best_params_1)

ClassTransformation(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05,
                                            max_delta_step=None, max_depth=3,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=25, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=8,
                                            reg_alpha=0.05, reg_lambda=0,
                                            scale_pos_weight=None,
                                            subsample=None, tree_method=None,
                                            use_label_encoder=False,
                                            validate_parameters=None,
                                            verbosity=None))

In [50]:
model_2.fit(X_train, y_train, treat_train)

/home/denis/MyCode/Git/Kaggle_CFT/env/lib/python3.10/site-packages/xgboost/core.py:499: FutureWarning: Pass `sample_weight` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[15:10:18] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=25, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=8,
              reg_alpha=0.05, reg_lambda=0, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [53]:
z_pred_2 = model_2.predict_proba(X_test)

In [54]:
z_pred_2

array([[0.5178831 , 0.48211688],
       [0.40170228, 0.5982977 ],
       [0.2849899 , 0.7150101 ],
       ...,
       [0.22296774, 0.77703226],
       [0.33747983, 0.6625202 ],
       [0.44414663, 0.55585337]], dtype=float32)

In [65]:
y_pred_2 = 2 * z_pred_2[:, 1] - 1
y_pred_2

array([-0.03576624,  0.19659543,  0.4300202 , ...,  0.5540645 ,
        0.32504034,  0.11170673], dtype=float32)

In [81]:
metrics.qini_auc_score(y_test, y_pred_2, treat_test)

-0.008232586589097209

In [78]:
pd.DataFrame({'y_pred_1':y_pred_1, 'z_pred_2': z_pred_2[:,1], 'y_pred_2': y_pred_2}).describe()

,y_pred_1,z_pred_2,y_pred_2
count,42009.000000,42009.000000,42009.000000
mean,0.025190,0.598884,0.197768
std,0.045711,0.147022,0.294037
min,-0.127392,0.320913,-0.358174
25%,0.013208,0.488364,-0.023272
50%,0.019734,0.609092,0.218184
75%,0.030564,0.730671,0.461342
max,0.389614,0.818999,0.637998


# Conclusion. Maybe I do not understand something but trying to predict z does not work.